In [18]:
import argparse
import os
import pandas as pd
import snowflake.connector
import json
import boto3
import xxhash
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from sklearn.metrics import roc_auc_score
import numpy as np
from datetime import datetime, timedelta
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score

from typing import Dict

from tqdm import tqdm
from code.snowflake_utils import connect


sf_account_id = "lnb99345.us-east-1"
sf_secret_id = "snowflake_credentials"
warehouse = "XSMALL"
database = "PRODUCTION"
schema = "SIGNALS"


In [2]:
ctx = connect(secret_id=sf_secret_id,
              account=sf_account_id, 
              warehouse=warehouse,
              database=database,
              schema=schema, 
              protocol="https",
              region="us-east-1",
              profile_name="ml-staging-admin")
ctx

sf_user=MGAIDUK, sf_password=****, sf_account=lnb99345.us-east-1, sf_warehouse=XSMALL, sf_database=PRODUCTION, sf_schema=SIGNALS, sf_protocol=https


In [6]:
def collect_dataset(ctx: snowflake.connector.SnowflakeConnection, input: str) -> pd.DataFrame:
    # train on a 7-day lookback window
    filter_time = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-%d')
    # Collect dataset
    sql = f"""
    select * from {input}
    where min_timestamp >= '{filter_time}'
    """
    df = pd.read_sql(sql, ctx)
    return df

df = collect_dataset(ctx, input="TRAIN_DATASET_V2")
df.shape, df

/tmp/ipykernel_172561/86426041.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, ctx)


((412844, 168),
                                       USERID                     MEDIAID  \
 0       305d6f1c-3b83-4229-9272-8f3cc1e03207  01HN10AD812E1TW15NN3T6JAW6   
 1       aa5a25e3-8325-4731-acd9-14d453187341  01HHM78CDZVDDTNB01TMCS9JY2   
 2       05d441ba-4b95-406b-b186-246904b99a8a  01HB2PVRC6EDWDKT69NY3KQGW9   
 3       4923309a-0596-4059-ad37-9804fddb92ec  01HMM2ZH3XX4QFVPX4D4M5F469   
 4       0831e5e2-0e37-444e-84d3-cdf1f6f02203  01HM4CXYNMJDAZ7R1G6H4NBSGT   
 ...                                      ...                         ...   
 412839  14ee69a4-cada-411f-aff1-439436da6ff5  01H18GQHXP39ZXYM72AD56HG79   
 412840  5588bfd9-e5e7-4610-b466-131b99e8120b  01HF119R260RAQ8K98T6WKYK25   
 412841  612ac0d4-9976-4e5c-aeb9-559db6f51826  01HEJHYAG9T9T5VQ3R4EV3QEDE   
 412842  eb4e3d65-5678-42b5-97a6-e52bdc5d320c  01HMK0CHG2Z80JNHV1B1FA4ZC7   
 412843  a3734971-389e-40d5-841f-b7608d32dd8a  01HP9Y8JC4YD75F6YDYCE5J32F   
 
                               MEDIATAKENBYID  \
 0       

In [9]:
list(df.columns)

['USERID',
 'MEDIAID',
 'MEDIATAKENBYID',
 'REQUESTID',
 'NVIEWS',
 'TIMESPENT',
 'MIN_TIMESTAMP',
 'NREACTIONS',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1M',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_5M',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_15M',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1H',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_6H',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1D',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_7D',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_30D',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1M',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_5M',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_15M',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1H',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_6H',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1D',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_7D',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_30D',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_1M',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_5M',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_15M',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_1H',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_6H',
 'TAKENBY_CFMEDIADWELLTIMES

In [10]:
df["HAS_REACTIONS"] = df["NREACTIONS"] > 0
df["HAS_REACTIONS"]

0         False
1         False
2         False
3         False
4         False
          ...  
412839    False
412840    False
412841    False
412842    False
412843    False
Name: HAS_REACTIONS, Length: 412844, dtype: bool

In [11]:
yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

val_df = df[df["MIN_TIMESTAMP"] > yesterday]
train_df = df[df["MIN_TIMESTAMP"] <= yesterday]
train_df.shape, val_df.shape

((185681, 169), (227163, 169))

In [13]:
features = ['TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1M','TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_5M','TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_15M','TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1H','TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_6H','TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1D','TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_7D','TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_30D','TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1M','TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_5M','TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_15M','TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1H','TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_6H','TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1D','TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_7D','TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_30D','TAKENBY_CFMEDIADWELLTIMESIGNAL_1M','TAKENBY_CFMEDIADWELLTIMESIGNAL_5M','TAKENBY_CFMEDIADWELLTIMESIGNAL_15M','TAKENBY_CFMEDIADWELLTIMESIGNAL_1H','TAKENBY_CFMEDIADWELLTIMESIGNAL_6H','TAKENBY_CFMEDIADWELLTIMESIGNAL_1D','TAKENBY_CFMEDIADWELLTIMESIGNAL_7D','TAKENBY_CFMEDIADWELLTIMESIGNAL_30D','TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1M','TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_5M','TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_15M','TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1H','TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_6H','TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1D','TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_7D','TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_30D','TAKENBY_CFMEDIAVIEWEDSIGNAL_1M','TAKENBY_CFMEDIAVIEWEDSIGNAL_5M','TAKENBY_CFMEDIAVIEWEDSIGNAL_15M','TAKENBY_CFMEDIAVIEWEDSIGNAL_1H','TAKENBY_CFMEDIAVIEWEDSIGNAL_6H','TAKENBY_CFMEDIAVIEWEDSIGNAL_1D','TAKENBY_CFMEDIAVIEWEDSIGNAL_7D','TAKENBY_CFMEDIAVIEWEDSIGNAL_30D','MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1M','MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_5M','MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_15M','MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1H','MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_6H','MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1D','MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_7D','MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_30D','MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1M','MEDIA_CFMEDIAADDEDREACTIONSIGNAL_5M','MEDIA_CFMEDIAADDEDREACTIONSIGNAL_15M','MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1H','MEDIA_CFMEDIAADDEDREACTIONSIGNAL_6H','MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1D','MEDIA_CFMEDIAADDEDREACTIONSIGNAL_7D','MEDIA_CFMEDIAADDEDREACTIONSIGNAL_30D','MEDIA_CFMEDIADWELLTIMESIGNAL_1M','MEDIA_CFMEDIADWELLTIMESIGNAL_5M','MEDIA_CFMEDIADWELLTIMESIGNAL_15M','MEDIA_CFMEDIADWELLTIMESIGNAL_1H','MEDIA_CFMEDIADWELLTIMESIGNAL_6H','MEDIA_CFMEDIADWELLTIMESIGNAL_1D','MEDIA_CFMEDIADWELLTIMESIGNAL_7D','MEDIA_CFMEDIADWELLTIMESIGNAL_30D','MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1M','MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_5M','MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_15M','MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1H','MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_6H','MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1D','MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_7D','MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_30D','MEDIA_CFMEDIAVIEWEDSIGNAL_1M','MEDIA_CFMEDIAVIEWEDSIGNAL_5M','MEDIA_CFMEDIAVIEWEDSIGNAL_15M','MEDIA_CFMEDIAVIEWEDSIGNAL_1H','MEDIA_CFMEDIAVIEWEDSIGNAL_6H','MEDIA_CFMEDIAVIEWEDSIGNAL_1D','MEDIA_CFMEDIAVIEWEDSIGNAL_7D','MEDIA_CFMEDIAVIEWEDSIGNAL_30D','USER_CFMEDIAADDEDCOMMENTSIGNAL_1M','USER_CFMEDIAADDEDCOMMENTSIGNAL_5M','USER_CFMEDIAADDEDCOMMENTSIGNAL_15M','USER_CFMEDIAADDEDCOMMENTSIGNAL_1H','USER_CFMEDIAADDEDCOMMENTSIGNAL_6H','USER_CFMEDIAADDEDCOMMENTSIGNAL_1D','USER_CFMEDIAADDEDCOMMENTSIGNAL_7D','USER_CFMEDIAADDEDCOMMENTSIGNAL_30D','USER_CFMEDIAADDEDREACTIONSIGNAL_1M','USER_CFMEDIAADDEDREACTIONSIGNAL_5M','USER_CFMEDIAADDEDREACTIONSIGNAL_15M','USER_CFMEDIAADDEDREACTIONSIGNAL_1H','USER_CFMEDIAADDEDREACTIONSIGNAL_6H','USER_CFMEDIAADDEDREACTIONSIGNAL_1D','USER_CFMEDIAADDEDREACTIONSIGNAL_7D','USER_CFMEDIAADDEDREACTIONSIGNAL_30D','USER_CFMEDIADWELLTIMESIGNAL_1M','USER_CFMEDIADWELLTIMESIGNAL_5M','USER_CFMEDIADWELLTIMESIGNAL_15M','USER_CFMEDIADWELLTIMESIGNAL_1H','USER_CFMEDIADWELLTIMESIGNAL_6H','USER_CFMEDIADWELLTIMESIGNAL_1D','USER_CFMEDIADWELLTIMESIGNAL_7D','USER_CFMEDIADWELLTIMESIGNAL_30D','USER_CFMEDIATAPPEDTAKENBYSIGNAL_1M','USER_CFMEDIATAPPEDTAKENBYSIGNAL_5M','USER_CFMEDIATAPPEDTAKENBYSIGNAL_15M','USER_CFMEDIATAPPEDTAKENBYSIGNAL_1H','USER_CFMEDIATAPPEDTAKENBYSIGNAL_6H','USER_CFMEDIATAPPEDTAKENBYSIGNAL_1D','USER_CFMEDIATAPPEDTAKENBYSIGNAL_7D','USER_CFMEDIATAPPEDTAKENBYSIGNAL_30D','USER_CFMEDIAVIEWEDSIGNAL_1M','USER_CFMEDIAVIEWEDSIGNAL_5M','USER_CFMEDIAVIEWEDSIGNAL_15M','USER_CFMEDIAVIEWEDSIGNAL_1H','USER_CFMEDIAVIEWEDSIGNAL_6H','USER_CFMEDIAVIEWEDSIGNAL_1D','USER_CFMEDIAVIEWEDSIGNAL_7D','USER_CFMEDIAVIEWEDSIGNAL_30D','CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1M','CROSS_CFMEDIAADDEDCOMMENTSIGNAL_5M','CROSS_CFMEDIAADDEDCOMMENTSIGNAL_15M','CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1H','CROSS_CFMEDIAADDEDCOMMENTSIGNAL_6H','CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1D','CROSS_CFMEDIAADDEDCOMMENTSIGNAL_7D','CROSS_CFMEDIAADDEDCOMMENTSIGNAL_30D','CROSS_CFMEDIAADDEDREACTIONSIGNAL_1M','CROSS_CFMEDIAADDEDREACTIONSIGNAL_5M','CROSS_CFMEDIAADDEDREACTIONSIGNAL_15M','CROSS_CFMEDIAADDEDREACTIONSIGNAL_1H','CROSS_CFMEDIAADDEDREACTIONSIGNAL_6H','CROSS_CFMEDIAADDEDREACTIONSIGNAL_1D','CROSS_CFMEDIAADDEDREACTIONSIGNAL_7D','CROSS_CFMEDIAADDEDREACTIONSIGNAL_30D','CROSS_CFMEDIADWELLTIMESIGNAL_1M','CROSS_CFMEDIADWELLTIMESIGNAL_5M','CROSS_CFMEDIADWELLTIMESIGNAL_15M','CROSS_CFMEDIADWELLTIMESIGNAL_1H','CROSS_CFMEDIADWELLTIMESIGNAL_6H','CROSS_CFMEDIADWELLTIMESIGNAL_1D','CROSS_CFMEDIADWELLTIMESIGNAL_7D','CROSS_CFMEDIADWELLTIMESIGNAL_30D','CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1M','CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_5M','CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_15M','CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1H','CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_6H','CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1D','CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_7D','CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_30D','CROSS_CFMEDIAVIEWEDSIGNAL_1M','CROSS_CFMEDIAVIEWEDSIGNAL_5M','CROSS_CFMEDIAVIEWEDSIGNAL_15M','CROSS_CFMEDIAVIEWEDSIGNAL_1H','CROSS_CFMEDIAVIEWEDSIGNAL_6H','CROSS_CFMEDIAVIEWEDSIGNAL_1D','CROSS_CFMEDIAVIEWEDSIGNAL_7D','CROSS_CFMEDIAVIEWEDSIGNAL_30D']
target = "HAS_REACTIONS"

In [14]:
dtrain = xgb.DMatrix(train_df[features], label=train_df[target])
dval = xgb.DMatrix(val_df[features], label=val_df[target])

In [23]:
params = {
    'objective': 'binary:logistic',  # Binary classification
    'max_depth': 6,                  # Depth of each tree
    'alpha': 1,                     # L1 regularization term on weights
    'learning_rate': 0.05,           # Step size shrinkage used in update to prevents overfitting
}
evals = [(dtrain, 'train'), (dval, 'val')]
eval_metric = ['error', 'auc']
bst = xgb.train(params, dtrain, num_boost_round=100, evals=evals)
y_pred = bst.predict(dval)
auc_score = roc_auc_score(val_df[target], y_pred)
print(f"Validation AUC: {auc_score:.3f}")

[0]	train-logloss:0.21110	val-logloss:0.21095
[1]	train-logloss:0.20588	val-logloss:0.20617
[2]	train-logloss:0.20104	val-logloss:0.20173
[3]	train-logloss:0.19651	val-logloss:0.19758
[4]	train-logloss:0.19228	val-logloss:0.19370
[5]	train-logloss:0.18832	val-logloss:0.19000
[6]	train-logloss:0.18451	val-logloss:0.18658
[7]	train-logloss:0.18090	val-logloss:0.18338
[8]	train-logloss:0.17762	val-logloss:0.18039
[9]	train-logloss:0.17455	val-logloss:0.17759
[10]	train-logloss:0.17168	val-logloss:0.17495
[11]	train-logloss:0.16896	val-logloss:0.17252
[12]	train-logloss:0.16642	val-logloss:0.17025
[13]	train-logloss:0.16405	val-logloss:0.16810
[14]	train-logloss:0.16177	val-logloss:0.16609
[15]	train-logloss:0.15964	val-logloss:0.16422
[16]	train-logloss:0.15769	val-logloss:0.16242
[17]	train-logloss:0.15582	val-logloss:0.16080
[18]	train-logloss:0.15407	val-logloss:0.15927
[19]	train-logloss:0.15245	val-logloss:0.15782
[20]	train-logloss:0.15090	val-logloss:0.15650
[21]	train-logloss:0.14